# AWS Postgres RDS Debug Notebook

This notebook allows to run queries manually against the AWS postgres DB.

# Prepare

In [1]:
import sys
import psycopg2
import psycopg2.extras
from urllib.parse import urlparse
import pandas as pd
from datetime import datetime
import os

# Import the database module
from ao.server.database_manager import DB

# Ensure we're using the remote PostgreSQL database
try:
    DB.switch_mode("remote")
except Exception as e:
    print(f"❌ Failed to switch to remote mode: {e}")

2025-12-18 19:45:58,267 - ACO - INFO - DatabaseManager initialized with backend: local
2025-12-18 19:45:58,269 - ACO - INFO - Switched to remote PostgreSQL database


## Check Database Connection

(You don't really need to run this. I just had to debug something once and wanted to keep it around)

In [2]:
# EMERGENCY NETWORK DIAGNOSIS - Run this first!
print("🚨 Basic network connectivity test...")

import socket
import subprocess
from urllib.parse import urlparse
from ao.common.constants import REMOTE_DATABASE_URL

if REMOTE_DATABASE_URL:
    parsed = urlparse(REMOTE_DATABASE_URL)
    host = parsed.hostname
    port = parsed.port or 5432
    
    print(f"Target: {host}:{port}")
    
    # Test 1: DNS resolution (should be instant)
    print(f"\n1️⃣ Testing DNS resolution for {host}...")
    try:
        import socket
        ip = socket.gethostbyname(host)
        print(f"✅ DNS resolved to: {ip}")
    except Exception as e:
        print(f"❌ DNS resolution failed: {e}")
        print("→ This means the hostname is invalid or DNS is broken")
    
    # Test 2: Ping test (basic connectivity)
    print(f"\n2️⃣ Testing ping to {host}...")
    try:
        result = subprocess.run(['ping', '-c', '2', host], 
                              capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            print("✅ Ping successful")
            # Extract timing info
            lines = result.stdout.split('\n')
            for line in lines:
                if 'time=' in line:
                    print(f"   {line.strip()}")
        else:
            print("❌ Ping failed")
            print(f"   Error: {result.stderr}")
    except subprocess.TimeoutExpired:
        print("❌ Ping timeout - host unreachable")
    except Exception as e:
        print(f"❌ Ping error: {e}")
    
    # Test 3: Telnet-style connection test (fastest way to test port)
    print(f"\n3️⃣ Testing port connectivity to {host}:{port}...")
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)  # Very short timeout
        result = sock.connect_ex((host, port))
        sock.close()
        
        if result == 0:
            print("✅ Port is reachable")
        else:
            print(f"❌ Port connection failed (error {result})")
            print("   → Database server may be down or port blocked")
    except Exception as e:
        print(f"❌ Socket error: {e}")
        
else:
    print("❌ No DATABASE_URL found")

# Test 4: General internet connectivity 
print(f"\n4️⃣ Testing general internet connectivity...")
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(3)
    result = sock.connect_ex(("8.8.8.8", 53))  # Google DNS
    sock.close()
    
    if result == 0:
        print("✅ Internet connectivity OK")
    else:
        print("❌ No internet connectivity")
        print("   → Check your network connection")
except Exception as e:
    print(f"❌ Internet test failed: {e}")

print(f"\n🔍 If all tests pass but db.get_conn() hangs:")
print("   → The PostgreSQL server is likely overloaded or in maintenance")
print("   → Check AWS RDS console for instance status")
print("   → Try connecting from a different network/location")

🚨 Basic network connectivity test...
Target: workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com:5432

1️⃣ Testing DNS resolution for workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com...
✅ DNS resolved to: 98.90.57.89

2️⃣ Testing ping to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com...
❌ Ping timeout - host unreachable

3️⃣ Testing port connectivity to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com:5432...
✅ Port is reachable

4️⃣ Testing general internet connectivity...
✅ Internet connectivity OK

🔍 If all tests pass but db.get_conn() hangs:
   → The PostgreSQL server is likely overloaded or in maintenance
   → Check AWS RDS console for instance status
   → Try connecting from a different network/location


## Show database Schema

In [3]:
# Show complete database schema information
print("📊 DATABASE SCHEMA OVERVIEW\n" + "="*50)

# Ensure we're using the remote PostgreSQL database
if DB.get_current_mode() != "remote":
    try:
        DB.switch_mode("remote")
    except Exception as e:
        print(f"❌ Failed to switch to remote mode: {e}")

# Only proceed if we're now using remote database
if DB.get_current_mode() == "remote":
    print("")
    
    # Get all tables in the database
    tables_query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public' 
        ORDER BY table_name
    """
    tables = DB.query_all(tables_query)
    
    if not tables:
        print("❌ No tables found in the database")
    else:
        print(f"✅ Found {len(tables)} tables in the database:\n")
        
        for table_info in tables:
            table_name = table_info['table_name']
            print(f"\n📋 Table: {table_name.upper()}")
            print("-" * 40)
            
            # Get columns for each table
            columns_query = """
                SELECT 
                    column_name,
                    data_type,
                    character_maximum_length,
                    is_nullable,
                    column_default
                FROM information_schema.columns 
                WHERE table_name = %s AND table_schema = 'public'
                ORDER BY ordinal_position
            """
            columns = DB.query_all(columns_query, (table_name,))
            
            # Display columns
            for col in columns:
                col_name = col['column_name']
                col_type = col['data_type']
                
                # Add length info if applicable
                if col['character_maximum_length']:
                    col_type += f"({col['character_maximum_length']})"
                
                # Add NULL/NOT NULL
                nullable = "NULL" if col['is_nullable'] == 'YES' else "NOT NULL"
                
                # Add default value if exists
                default = f" DEFAULT {col['column_default']}" if col['column_default'] else ""
                
                print(f"  • {col_name}: {col_type} {nullable}{default}")
            
            # Get primary key constraints
            pk_query = """
                SELECT column_name
                FROM information_schema.key_column_usage kcu
                JOIN information_schema.table_constraints tc 
                    ON kcu.constraint_name = tc.constraint_name
                WHERE tc.table_name = %s 
                    AND tc.constraint_type = 'PRIMARY KEY'
                    AND tc.table_schema = 'public'
                ORDER BY kcu.ordinal_position
            """
            pk_cols = DB.query_all(pk_query, (table_name,))
            
            if pk_cols:
                pk_names = [col['column_name'] for col in pk_cols]
                print(f"\n  🔑 Primary Key: ({', '.join(pk_names)})")
            
            # Get foreign key constraints
            fk_query = """
                SELECT
                    kcu.column_name,
                    ccu.table_name AS foreign_table_name,
                    ccu.column_name AS foreign_column_name
                FROM information_schema.table_constraints AS tc
                JOIN information_schema.key_column_usage AS kcu
                    ON tc.constraint_name = kcu.constraint_name
                JOIN information_schema.constraint_column_usage AS ccu
                    ON ccu.constraint_name = tc.constraint_name
                WHERE tc.constraint_type = 'FOREIGN KEY' 
                    AND tc.table_name = %s
                    AND tc.table_schema = 'public'
            """
            fk_cols = DB.query_all(fk_query, (table_name,))
            
            if fk_cols:
                print("\n  🔗 Foreign Keys:")
                for fk in fk_cols:
                    print(f"    • {fk['column_name']} → {fk['foreign_table_name']}.{fk['foreign_column_name']}")
            
            # Get unique constraints
            unique_query = """
                SELECT column_name
                FROM information_schema.key_column_usage kcu
                JOIN information_schema.table_constraints tc 
                    ON kcu.constraint_name = tc.constraint_name
                WHERE tc.table_name = %s 
                    AND tc.constraint_type = 'UNIQUE'
                    AND tc.table_schema = 'public'
            """
            unique_cols = DB.query_all(unique_query, (table_name,))
            
            if unique_cols:
                unique_names = [col['column_name'] for col in unique_cols]
                print(f"\n  ⭐ Unique Constraints: {', '.join(unique_names)}")
            
            # Get indexes
            index_query = """
                SELECT 
                    indexname,
                    indexdef
                FROM pg_indexes
                WHERE tablename = %s
                    AND schemaname = 'public'
            """
            indexes = DB.query_all(index_query, (table_name,))
            
            if indexes:
                print("\n  📍 Indexes:")
                for idx in indexes:
                    # Skip primary key index (already shown above)
                    if not idx['indexname'].endswith('_pkey'):
                        print(f"    • {idx['indexname']}")
            
            # Get row count
            count_query = f"SELECT COUNT(*) as count FROM {table_name}"
            count_result = DB.query_all(count_query)
            row_count = count_result[0]['count'] if count_result else 0
            print(f"\n  📈 Row Count: {row_count:,}")
    
    print("\n" + "="*50)
    print("✅ Schema overview complete")

2025-12-18 19:46:14,170 - ACO - DEBUG - Loaded PostgreSQL backend module


📊 DATABASE SCHEMA OVERVIEW



2025-12-18 19:46:15,745 - ACO - DEBUG - Database schema initialized
2025-12-18 19:46:15,745 - ACO - INFO - Initialized PostgreSQL connection pool to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com


✅ Found 5 tables in the database:


📋 Table: ATTACHMENTS
----------------------------------------
  • file_id: text NOT NULL
  • session_id: text NULL
  • line_no: integer NULL
  • content_hash: text NULL
  • file_path: text NULL
  • taint: text NULL

  🔑 Primary Key: (file_id)

  🔗 Foreign Keys:
    • session_id → experiments.session_id

  📍 Indexes:
    • attachments_content_hash_idx

  📈 Row Count: 0

📋 Table: EXPERIMENTS
----------------------------------------
  • session_id: text NOT NULL
  • parent_session_id: text NULL
  • graph_topology: text NULL
  • color_preview: text NULL
  • timestamp: timestamp without time zone NULL DEFAULT CURRENT_TIMESTAMP
  • cwd: text NULL
  • command: text NULL
  • environment: text NULL
  • code_hash: text NULL
  • name: text NULL
  • success: text NULL
  • notes: text NULL
  • log: text NULL
  • user_id: integer NULL

  🔑 Primary Key: (session_id)

  🔗 Foreign Keys:
    • parent_session_id → experiments.session_id
    • user_id → users.id

  ⭐ Un

## List Table Entries

### experiments table

In [4]:
# Get all experiments
experiments = DB.query_all(
    "SELECT session_id, name, timestamp, user_id FROM experiments ORDER BY timestamp DESC LIMIT 20"
)

if experiments:
    df_experiments = pd.DataFrame(experiments)
    print(f"Found {len(experiments)} experiments:")
    display(df_experiments)
else:
    print("No experiments found in database")

No experiments found in database


### llm_calls table

In [5]:
# Get recent LLM calls
llm_calls = DB.query_all(
    "SELECT session_id, node_id, api_type, timestamp FROM llm_calls ORDER BY timestamp DESC LIMIT 20"
)

if llm_calls:
    df_llm = pd.DataFrame(llm_calls)
    print(f"Found {len(llm_calls)} recent LLM calls:")
    display(df_llm)
else:
    print("No LLM calls found")

No LLM calls found


## users table

In [6]:
# Get all users
users = DB.query_all(
    "SELECT id, email, name, created_at, google_id FROM users"
)

if users:
    df_users = pd.DataFrame(users)
    print(f"Found {len(users)} users:")
    display(users)
else:
    print("No users found in database")

No users found in database


# DANGER ZONE: Drop tables and reset schema

In [7]:
# ---> Comment below out to run schema change. <---
# NOTE: Dropping a table deletes all its data and is irreversible.
from ao.server.database_manager import DB

print("🔧 RESETTING DATABASE SCHEMA TO MATCH postgres.py\n" + "="*50)

try:
    # Ensure we're using the remote PostgreSQL database
    if DB.get_current_mode() != "remote":
        DB.switch_mode("remote")
        print("1️⃣ Switched to remote PostgreSQL database")
    else:
        print("1️⃣ Already using remote PostgreSQL database")
    
    # Drop all existing tables in correct order (respecting foreign keys)
    print("\n2️⃣ Dropping existing tables...")
    DB.execute("DROP TABLE IF EXISTS attachments CASCADE")
    print("   ✅ Dropped attachments table")
    DB.execute("DROP TABLE IF EXISTS llm_calls CASCADE")
    print("   ✅ Dropped llm_calls table")
    DB.execute("DROP TABLE IF EXISTS experiments CASCADE")
    print("   ✅ Dropped experiments table")
    DB.execute("DROP TABLE IF EXISTS users CASCADE")
    print("   ✅ Dropped users table")
    
    # Create users table first (referenced by experiments)
    print("\n3️⃣ Creating tables with schema from postgres.py...")
    DB.execute(
        """
        CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY,
            google_id VARCHAR(255) NOT NULL UNIQUE,
            email VARCHAR(255) NOT NULL UNIQUE,
            name VARCHAR(255) NOT NULL,
            picture TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
    )
    print("   ✅ Created users table")
    
    # Create experiments table
    DB.execute(
        """
        CREATE TABLE IF NOT EXISTS experiments (
            session_id TEXT PRIMARY KEY,
            parent_session_id TEXT,
            graph_topology TEXT,
            color_preview TEXT,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            cwd TEXT,
            command TEXT,
            environment TEXT,
            code_hash TEXT,
            name TEXT,
            success TEXT CHECK (success IN ('', 'Satisfactory', 'Failed')),
            notes TEXT,
            log TEXT,
            user_id INTEGER,
            FOREIGN KEY (parent_session_id) REFERENCES experiments (session_id),
            FOREIGN KEY (user_id) REFERENCES users (id),
            UNIQUE (parent_session_id, name)
        )
        """
    )
    print("   ✅ Created experiments table")
    
    # Create llm_calls table
    DB.execute(
        """
        CREATE TABLE IF NOT EXISTS llm_calls (
            session_id TEXT,
            node_id TEXT,
            input TEXT,
            input_hash TEXT,
            input_overwrite TEXT,
            output TEXT,
            color TEXT,
            label TEXT,
            api_type TEXT,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (session_id, node_id),
            FOREIGN KEY (session_id) REFERENCES experiments (session_id)
        )
        """
    )
    print("   ✅ Created llm_calls table")
    
    # Create attachments table
    DB.execute(
        """
        CREATE TABLE IF NOT EXISTS attachments (
            file_id TEXT PRIMARY KEY,
            session_id TEXT,
            line_no INTEGER,
            content_hash TEXT,
            file_path TEXT,
            taint TEXT,
            FOREIGN KEY (session_id) REFERENCES experiments (session_id)
        )
        """
    )
    print("   ✅ Created attachments table")
    
    # Create indexes (exactly as in postgres.py)
    print("\n4️⃣ Creating indexes...")
    DB.execute(
        """
        CREATE INDEX IF NOT EXISTS attachments_content_hash_idx ON attachments(content_hash)
        """
    )
    print("   ✅ Created attachments_content_hash_idx")
    
    DB.execute(
        """
        CREATE INDEX IF NOT EXISTS original_input_lookup ON llm_calls(session_id, input_hash)
        """
    )
    print("   ✅ Created original_input_lookup index")
    
    DB.execute(
        """
        CREATE INDEX IF NOT EXISTS experiments_timestamp_idx ON experiments(timestamp DESC)
        """
    )
    print("   ✅ Created experiments_timestamp_idx")
    
    # Verify the schema
    print("\n5️⃣ Verifying schema...")
    
    # Check llm_calls output column type
    output_col = DB.query_one("""
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = 'llm_calls' AND column_name = 'output'
    """)
    
    # Check users table exists
    users_count = DB.query_one("""
        SELECT COUNT(*) as count FROM information_schema.tables 
        WHERE table_schema = 'public' AND table_name = 'users'
    """)
    if users_count and users_count['count'] > 0:
        print(f"   ✅ Users table exists")
    
    # Get table count
    table_count_result = DB.query_one("""
        SELECT COUNT(*) as count FROM information_schema.tables 
        WHERE table_schema = 'public'
    """)
    table_count = table_count_result['count'] if table_count_result else 0
    
    print(f"\n✅ SCHEMA RESET COMPLETE!")
    print(f"   • {table_count} tables created")
    print(f"   • All indexes created")
    print(f"   • Schema now matches postgres.py exactly")
    
except Exception as e:
    print(f"\n❌ Schema reset failed: {e}")
    print(f"   Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()

🔧 RESETTING DATABASE SCHEMA TO MATCH postgres.py
1️⃣ Already using remote PostgreSQL database

2️⃣ Dropping existing tables...
   ✅ Dropped attachments table
   ✅ Dropped llm_calls table
   ✅ Dropped experiments table
   ✅ Dropped users table

3️⃣ Creating tables with schema from postgres.py...
   ✅ Created users table
   ✅ Created experiments table
   ✅ Created llm_calls table
   ✅ Created attachments table

4️⃣ Creating indexes...
   ✅ Created attachments_content_hash_idx
   ✅ Created original_input_lookup index
   ✅ Created experiments_timestamp_idx

5️⃣ Verifying schema...
   ✅ Users table exists

✅ SCHEMA RESET COMPLETE!
   • 5 tables created
   • All indexes created
   • Schema now matches postgres.py exactly
